<a href="https://colab.research.google.com/github/kimminseo9/Realtime-headline-news_and_market-reaction/blob/main/%EC%8B%A4%EC%8B%9C%EA%B0%84_%ED%97%A4%EB%93%9C%EB%9D%BC%EC%9D%B8_%EB%89%B4%EC%8A%A4%EC%99%80_%EC%8B%9C%EC%9E%A5_%EB%B0%98%EC%9D%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 라이브러리 설치 임포트

In [1]:
!pip install konlpy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 15.9 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import json
from IPython.display import Image, display, Markdown

## 네이버 뉴스 크롤링

In [3]:
def get_soup_obj(news_link):
  headers = {'User-Agent': 'Mozilla/5.0'}
  response = requests.get(news_link, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  return soup

In [4]:
def get_top3_news_info(sec, sid):
  default_img = "https://search.naver.com/search.naver?where=image$sm=tab_jum$query=naver#"
  sec_url = "https://news.naver.com/section/%s" % sid
  print("section url : ", sec_url)

  news_list3 = []
  soup = get_soup_obj(sec_url)

  lis3 = soup.find_all("li", class_="sa_item _SECTION_HEADLINE", limit=3)
  for li in lis3:
    title_tag = li.find('strong', class_="sa_text_strong")
    a_tag = li.select_one('a.sa_text_title')
    media_tag = li.find('div', class_='sa_text_info_left')
    img_tag = li.find('img')


    news_info = {
        'title': title_tag.get_text(strip=True) if title_tag else None,
        'media_com': media_tag.get_text(strip=True) if media_tag else None,
        'news_link': a_tag['href'] if a_tag and 'href' in a_tag.attrs else None,
        'img_src': (img_tag.get('data-src') or img_tag.get('src')) if img_tag else None
    }
    news_list3.append(news_info)

  return news_list3

In [5]:
def get_article_contents(news_link):
    soup = get_soup_obj(news_link)

    # 다양한 구조 대응
    article = soup.find('article', id='dic_area') \
           or soup.find('div', id='newsct_article') \
           or soup.find('div', class_='article_body')

    if not article:
        return "본문을 찾을 수 없습니다."

    content = article.get_text(separator='\n', strip=True)

    return content

In [6]:
summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")

def naver_news_top3():
  news_dic = dict()
  sections = ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']
  section_ids = ['100', '101', '102', '103', '104', '105']

  for sec, sid in zip(sections, section_ids):
    news_info = get_top3_news_info(sec, sid)

    for news in news_info:
      news_link = news['news_link']
      news_contents = get_article_contents(news_link)
      news['news_contents'] = news_contents

      try:
        summary_output = summarizer(news_info['news_contents'], max_length=128, min_length=30, do_sample=False)
        snews_contents = summary_output[0]['summary_text']
      except Exception as e:
        sentences = news_contents.split('.')
        if len(sentences) > 3:
          snews_contents = '.'.join(sentences[:3])+'.'
        else:
          snews_contents = news_contents

      news['snews_contents'] = snews_contents

    news_dic[sec] = news_info

  return news_dic

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


In [7]:
news_dic = naver_news_top3()
news_dic

section url :  https://news.naver.com/section/100
section url :  https://news.naver.com/section/101
section url :  https://news.naver.com/section/102
section url :  https://news.naver.com/section/103
section url :  https://news.naver.com/section/104
section url :  https://news.naver.com/section/105


{'pol': [{'title': '정청래 "통쾌하게, 속 시원한 강력한 개혁 민주당 대표 될 것"',
   'media_com': '파이낸셜뉴스',
   'news_link': 'https://n.news.naver.com/mnews/article/014/0005386158',
   'img_src': 'https://mimgnews.pstatic.net/image/origin/014/2025/08/02/5386158.jpg?type=nf220_150',
   'news_contents': '더불어민주당 제2차 임시전국당원대회\n정청래 당 대표 후보 정견 발표\n더불어민주당 정청래 당 대표 후보가 2일 고양 킨텍스에서 열린 전국당원대회에서 정견발표를 하고 있다. 연합뉴스\n【파이낸셜뉴스 고양(경기)=최종근 송지원 김형구 기자】\n정청래 더불어민주당 당 대표 후보는 2일 경기 킨텍스에서 열린 제2차 임시전국당원대회 정견 발표에서 "법제사법위원장때처럼 통쾌하게, 효능감있게, 속 시원하게 당 대표를 하겠다"고 밝혔다.\n정 후보는 이날 경기 킨텍스에서 열린 민주당 당원대회에서 "첫째도 개혁, 둘째도 개혁, 셋째도 개혁, 강력한 개혁 당 대표"라는 점을 강조하면서 "민심을 이기는 정권이 없고, 당심을 이기는 정권은 없다. 강력한 개혁 당 대표 정청래의 손을 잡아달라"고 말했다.\n그러면서 "윤석열을 탄핵하고 파면하고 우리는 대선을 이겼다. 이재명 대통령을 만들었다"면서 "최전방 공격수로 개혁의 골을 넣겠다. 올라운드 플레이어로 당정대 원팀 플레이 감독으로 당을 잘 이끌겠다"고 했다.\n정 후보는 "검찰개혁, 언론개혁, 사법개혁은 폭풍처럼 몰아쳐서 전광석화처럼 해치우겠다"면서 "추석 전 귀향길 자동차 라디오 뉴스에서 검찰청이 폐지됐다는 소식을 반드시 들려드리겠다. 3개월 안에 개혁 입법을 끝내고, 개혁 고속도로를 이재명 정부의 자동차가 민생을 싣고 쌩쌩 달릴 수 있도록 하겠다"고 강조했다.\n협치, 통합 보다는 싸움을 통해 강력한 개혁을 이끌겠다는 의지도 보였다. 정

## 댓글 스크래핑

In [8]:
def extract_press_article_id(news_link):
    match = re.search(r'oid=(\d+)&aid=(\d+)', news_link)
    if match:
        return match.groups()

    match = re.search(r'/article/(\d{3})/(\d+)', news_link)

    if match:
        return match.groups()

    return None, None


def get_comment_count(press_id, article_id):
    url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
    params = {
        'ticket': 'news',
        'templateId': 'default',
        'pool': 'cbox5',
        'lang': 'ko',
        'country': 'KR',
        'objectId': f"news{press_id},{article_id}",
        'pageSize': 1,
        'indexSize': 10
    }
    headers = {
        'Referer': f'https://news.naver.com/main/read.naver?oid={press_id}&aid={article_id}',
        'User-Agent': 'Mozilla/5.0'
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        try:
            json_str = response.text[response.text.find('(')+1:-2]
            data = json.loads(json_str)
            return data['result']['count']['total']
        except Exception as e:
            print(f"[❌ JSON 파싱 오류] {press_id}, {article_id}: {e}")
            return None
    else:
        print(f"[❌ 요청 실패] status_code={response.status_code}, press_id={press_id}, article_id={article_id}")
        return None

In [9]:
def count_comments(news_dic, sec):
  for article in articles:
      news_link = article.get('news_link')
      press_id, article_id = extract_press_article_id(news_link)
      if press_id and article_id:
          count_comment = get_comment_count(press_id, article_id)
      else:
          count_comment = None
  return count_comment

In [10]:
def get_comments_only(news_link):
    press_id, article_id = extract_press_article_id(news_link)
    if not press_id or not article_id:
        return []

    object_id = f"news{press_id},{article_id}"
    referer_url = f"https://news.naver.com/main/read.naver?oid={press_id}&aid={article_id}"
    url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
    headers = {
        'Referer': referer_url,
        'User-Agent': 'Mozilla/5.0'
    }

    comments = []
    page = 1
    while True:
        params = {
            'ticket': 'news',
            'templateId': 'default',
            'pool': 'cbox5',
            'lang': 'ko',
            'country': 'KR',
            'objectId': object_id,
            'pageSize': 100,
            'indexSize': 10,
            'page': page
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            break

        try:
            json_str = response.text[response.text.find('(')+1:-2]
            data = json.loads(json_str)
            comment_list = data['result'].get('commentList', [])
            if not comment_list:
                break
            for c in comment_list:
                comments.append(c['contents'])

            page += 1
            time.sleep(0.2)

        except:
            break

    return comments

In [11]:
def collect_all_comments_only(news_dic, sec):
  all_comments_list = []
  for article in articles:
      news_link = article.get('news_link')
      comments = get_comments_only(news_link)
      all_comments_list.extend(comments)
  return pd.DataFrame({'comment': all_comments_list})

In [12]:
def comment_emotion(df_all_comments, model, tfidvect):
  def text_cleaning(text):
    text = re.sub(r"<.*?>|&[^;]+;", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^가-힣a-zA-Z0-9 ]", "", text)
    return text

  def get_pos(x):
    okt = Okt()
    pos = okt.pos(x)
    return ['{}/{}'.format(word, tag) for word, tag in pos if tag not in ['Josa', 'Eomi', 'Punctuation']]

  df_all_comments['cleaned'] = df_all_comments['comment'].apply(text_cleaning)
  df_all_comments['pos'] = df_all_comments['cleaned'].apply(get_pos)

  X = tfidvect.transform(df_all_comments['pos'])

  preds = model.predict(X)

  df_all_comments['pred'] = preds
  df_all_comments['emotion'] = df_all_comments['pred'].apply(
      lambda x: '긍정' if x==1 else('부정' if x==0 else '모호함')
      )

  return df_all_comments[['comment', 'emotion']]

In [13]:
def emotion_ratio(result_df):
  total = len(result_df)

  emotion_ratio = (
      result_df['emotion']
      .value_counts(normalize=True)
      .mul(100)
      .round(2)
      .to_dict()
  )

  return emotion_ratio

In [15]:
def identity_tokenizer(x): return x
def identity_preprocessor(x): return x
model = joblib.load('sentiment_model.pkl')
tfidvect = joblib.load('tfidf_vectorizer.pkl')

sections = ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']

for sec in sections:
    print(f"\n📂 섹션: {sec}")
    articles = news_dic.get(sec, [])

    for i, article in enumerate(articles):
        news_link = article.get('news_link')
        news_title = article.get('title')
        print(f" 📰 기사 {i+1}: {news_title}")

        comments = get_comments_only(news_link)
        comment_count = len(comments)

        if comment_count >= 5:
            print(f" 🥎 댓글 수가 {comment_count}개로 충분합니다. 댓글 분석을 시작합니다.")

            df_all_comments = collect_all_comments_only(news_dic, sec)
            result_df = comment_emotion(df_all_comments, model, tfidvect)

            emotion_ratios = emotion_ratio(result_df)
            for emotion, ratio in emotion_ratios.items():
              print(f"해당 기사에 대해 {emotion} 반응은 {ratio:.2f}%입니다.")


        else:
            print(f" ⚾️ 댓글 수가 {comment_count}개로 부족하여, 시장 반응을 확인하기 어렵습니다.")
        print('\n')


📂 섹션: pol
 📰 기사 1: 정청래 "통쾌하게, 속 시원한 강력한 개혁 민주당 대표 될 것"
 🥎 댓글 수가 37개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 61.02%입니다.
해당 기사에 대해 부정 반응은 33.90%입니다.
해당 기사에 대해 긍정 반응은 5.08%입니다.


 📰 기사 2: 영남 공략에 '난동사태' 서부지법 방문…국힘 '당권 5파전' 개막(종합)
 🥎 댓글 수가 21개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 61.02%입니다.
해당 기사에 대해 부정 반응은 33.90%입니다.
해당 기사에 대해 긍정 반응은 5.08%입니다.


 📰 기사 3: K2전차 폴란드 2차수출 무보·수출입銀 7조 지원
 ⚾️ 댓글 수가 1개로 부족하여, 시장 반응을 확인하기 어렵습니다.



📂 섹션: eco
 📰 기사 1: 관세에 증세, 노란봉투법, 더 센 상법까지..재계, 사회적 대화 촉구
 🥎 댓글 수가 81개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 부정 반응은 48.78%입니다.
해당 기사에 대해 모호함 반응은 46.34%입니다.
해당 기사에 대해 긍정 반응은 4.88%입니다.


 📰 기사 2: 악성 미분양 급증한 부산…초고가 분양은 청약 열기
 ⚾️ 댓글 수가 1개로 부족하여, 시장 반응을 확인하기 어렵습니다.


 📰 기사 3: 주유소 평균 휘발윳값 1,667.7원…경유 2주째 올라
 ⚾️ 댓글 수가 0개로 부족하여, 시장 반응을 확인하기 어렵습니다.



📂 섹션: soc
 📰 기사 1: “김 여사 친분 강조…청탁성 투자” 영장 기재
 ⚾️ 댓글 수가 0개로 부족하여, 시장 반응을 확인하기 어렵습니다.


 📰 기사 2: 남편 신체 훼손한 아내·가담한 사위 구속 기로
 🥎 댓글 수가 100개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 51.72%입니다.
해당 기사에 대해 부정 반응은 44.83%입니다.
해당 기사에 대해 긍정 반응은 3.4

## 섹션 선택 및 헤드라인 기사 추출

In [21]:
sections = ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']
print("사용 가능한 섹션:", sections)

selected_sec = input("검색할 섹션을 입력하세요: ").strip()

if selected_sec not in sections:
    print("❌ 유효하지 않은 섹션입니다. 다시 실행해주세요.")
else:
    print(f"✅ 선택된 섹션: {selected_sec}")
    articles = news_dic.get(selected_sec, [])

    for i, article in enumerate(articles):
        news_link = article.get('news_link')
        news_title = article.get('title')
        img_url = article['img_src']
        summary = article['snews_contents'].replace('\n', '\n\n')
        print(f"📰 기사 {i+1}")
        display(Image(url=img_url))
        display(Markdown(f"### 📰 {article['title']}\n\n{summary}"))

        comments = get_comments_only(news_link)
        comment_count = len(comments)

        if comment_count >= 5:
            print(f" 🥎 댓글 수가 {comment_count}개로 충분합니다. 댓글 분석을 시작합니다.")

            df_all_comments = collect_all_comments_only(news_dic, sec)
            result_df = comment_emotion(df_all_comments, model, tfidvect)

            emotion_ratios = emotion_ratio(result_df)
            for emotion, ratio in emotion_ratios.items():
              print(f"해당 기사에 대해 {emotion} 반응은 {ratio:.2f}%입니다.")


        else:
            print(f" ⚾️ 댓글 수가 {comment_count}개로 부족하여, 시장 반응을 확인하기 어렵습니다.")
        print('\n')

사용 가능한 섹션: ['pol', 'eco', 'soc', 'cul', 'wor', 'sci']
검색할 섹션을 입력하세요: eco
✅ 선택된 섹션: eco
📰 기사 1


### 📰 관세에 증세, 노란봉투법, 더 센 상법까지..재계, 사회적 대화 촉구

[앵커]

미국의 관세 인상에 이어 국내에선 법인세 인상과 노란봉투법, 더 센 상법이 입법 절차를 밟고 있습니다.

재계는 일제히 우려를 나타내고 사회적 대화를 통한 협의를 촉구하고 있습니다.

이승은 기자가 보도합니다.

 🥎 댓글 수가 87개로 충분합니다. 댓글 분석을 시작합니다.
해당 기사에 대해 모호함 반응은 48.86%입니다.
해당 기사에 대해 부정 반응은 46.59%입니다.
해당 기사에 대해 긍정 반응은 4.55%입니다.


📰 기사 2


### 📰 악성 미분양 급증한 부산…초고가 분양은 청약 열기

▲ 2010년 1월∼2025년 6월까지 부산 준공 후 미분양

공사가 끝났지만 팔리지 않아 '악성 미분양'으로 분류되는 '준공 후 미분양 주택' 물량이 부산에서 14년 11개월 만에 최고치를 기록했습니다.

오늘(2일) 국가통계포털에 따르면 올해 6월 말 기준 부산의 준공 후 미분양 주택은 2천663 가구로 집계됐습니다.

올해 1월 2천268 가구로 2천 가구대에 진입한 뒤 꾸준히 증가세를 이어오고 있습니다.

 ⚾️ 댓글 수가 1개로 부족하여, 시장 반응을 확인하기 어렵습니다.


📰 기사 3


### 📰 주유소 평균 휘발윳값 1,667.7원…경유 2주째 올라

이번 주 국내 주유소 휘발유와 경유 가격이 모두 올랐습니다.

오늘(2일) 한국석유공사 유가정보시스템 오피넷에 따르면 7월 다섯째 주 전국 주유소 휘발유 평균 판매가는 지난주보다 L당 0.3원 오른 1,667.

 ⚾️ 댓글 수가 0개로 부족하여, 시장 반응을 확인하기 어렵습니다.


